In [12]:
import os
import sys
import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=15,10
print("Matplotlib:{}".format(mpl.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

import keras
print("Keras:{}".format(keras.__version__))

current_path = os.getcwd()
base_dir = os.path.dirname(current_path)

if not base_dir in sys.path:
    sys.path.append(base_dir)

print(sys.path)

%reload_ext autoreload
%autoreload 2
import datasetslib
from datasetslib import util as dsu
from datasetslib import nputil

datasetslib.datasets_root = os.path.join(os.getcwd(), 'datasets')
models_root = os.path.join(os.getcwd(), 'models')


from datasetslib.imagenet import imageNet

# 注意：scipy库出现："cannot import name 'imread'"， 需要scipy版本1.2之下




    

NumPy:1.16.2
Matplotlib:2.2.2
TensorFlow:1.13.1
Keras:2.2.4
['C:\\Work\\pypath', 'c:\\python36\\python36.zip', 'c:\\python36\\DLLs', 'c:\\python36\\lib', 'c:\\python36', '', 'c:\\python36\\lib\\site-packages', 'c:\\python36\\lib\\site-packages\\setuptools-36.5.0-py3.6.egg', 'c:\\python36\\lib\\site-packages\\win32', 'c:\\python36\\lib\\site-packages\\win32\\lib', 'c:\\python36\\lib\\site-packages\\Pythonwin', 'c:\\python36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\wj_di\\.ipython', 'C:\\Work\\github\\tensorflow_learning']


In [23]:
def disp(images,id2label=None,probs=None,n_top=5,scale=False):
    if scale:
        imgs = np.abs(images + np.array([[[[123.68, 116.78, 103.94]]]]))/255.0
    else:
        imgs = images

    ids={}
    for j in range(len(images)):
        if scale:
            plt.figure(figsize=(5,5))
            plt.imshow(imgs[j])
        else:
            plt.imshow(imgs[j].astype(np.uint8) )
        plt.show()
        if probs is not None:
            ids[j] = [i[0] for i in sorted(enumerate(-probs[j]), key=lambda x:x[1])]
            for k in range(n_top):
                id = ids[j][k]
                print('Probability {0:1.2f}% of [{1:}]'
                      .format(100*probs[j,id],id2label[id]))


In [27]:
# Load the dataset
from datasetslib.coco import coco_animals

coco = coco_animals()
x_train_files, y_train, x_val_files, x_val = coco.load_data()

#x_test = [x_train_files[0],x_train_files[245]]
x_test = [x_val_files[25*x] for x in range(8)]
print('{}'.format(x_test))


Already exists: C:\Work\github\tensorflow_learning\ch12\datasets\coco-animals\coco-animals.zip
['C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\horse\\COCO_train2014_000000005903.jpg', 'C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\dog\\COCO_train2014_000000114469.jpg', 'C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\zebra\\COCO_train2014_000000303233.jpg', 'C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\dog\\COCO_train2014_000000299115.jpg', 'C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\cat\\COCO_train2014_000000379360.jpg', 'C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\horse\\COCO_train2014_000000183231.jpg', 'C:\\Work\\github\\tensorflow_learning\\ch12\\datasets\\coco-animals\\coco-animals\\train\\bird\\COCO_train2014_000000

In [28]:
images=np.array([mpimg.imread(x) for x in x_test])
disp(images)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'JpegImageFile'

In [ ]:
from tensorflow.contrib import slim
from tensorflow.contrib.slim.nets import vgg


inet = imageNet()
inet.load_data(n_classes=1000)

model_name = 'vgg_16'
model_url = 'http://download.tensorflow.org/models/'
model_files = ['vgg_16_2016_08_28.tar.gz']
model_home = os.path.join(models_root, model_name)

dsu.download_dataset(source_url=model_url, 
                    source_files=model_files,
                    dest_dir=model_home,
                    force=False,
                    extract=True)


image_height = vgg.vgg_16.default_image_size
image_width = vgg.vgg_16.default_image_size

tf.reset_default_graph()

x_p = tf.placeholder(shape=(None, image_height, image_width, 3),
                    dtype=tf.float32, name='x_p')

with slim.arg_scope(vgg.vgg_arg_scope()):
    logits, _ = vgg.vgg_16(x_p, num_classes=inet.n_classes, is_training=False)
   
probabilities = tf.nn.softmax(logits)

init = slim.assign_from_checkpoint_fn(os.path.join(model_home, '{}.ckpt'.format(model_name)))

with tf.Session() as tfs:
    init(tfs)
    probs = tfs.run([probibilities], feed_dict={x_p: images_test})
    probs = probs[0]
    

disp(images_test, id2label=inet.id2label, probs=probs, scale=True)

                    